<a href="https://colab.research.google.com/github/amrit2565/SeminarProject/blob/main/MovieRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving tmdb_5000_credits.csv to tmdb_5000_credits (1).csv
Saving tmdb_5000_movies.csv to tmdb_5000_movies (1).csv
User uploaded file "tmdb_5000_credits.csv" with length 40044293 bytes
User uploaded file "tmdb_5000_movies.csv" with length 5698602 bytes


In [ ]:
import numpy as np
import pandas as pd
import ast
movies=pd.read_csv('tmdb_5000_movies.csv') #reading the uploaded movies dataset
credits=pd.read_csv('tmdb_5000_credits.csv') #reading the uploaded credits dataset
movies=movies.merge(credits,on='title') #Merged the data of movies and credits on the basis of title
#We shall keep selective columns for our merged dataset
#Rest of the columns would be ignored as of now
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']] #Columns of dataset we shall be working on
#movies.head()
movies.dropna(inplace=True) #Using this we have removed data which don't have proper overview
#Also none of the data is duplicate of any other data

def convert(obj):  #This function has been created to get the proper genre keywords in a list and that list shall be used as genre
  L=[]
  for i in ast.literal_eval(obj):  #ast has been imprted to get list data as list and not as string. The above function was throwing error 
    L.append(i['name'])
  return L

movies['genres']=movies['genres'].apply(convert) #Using the above function we shall get proper Genre of each movies
#Similar thing shall be applied on Keywords as well
movies['keywords']=movies['keywords'].apply(convert)
#For cast we shall take in account the leading three casts and rest shall be ignored
#Function below has been created for the same
def convert3(obj): 
  L=[]
  counter=0
  for i in ast.literal_eval(obj):  
    if counter!=3:
     L.append(i['name'])
     counter+=1
    else:
      break

  return L
movies['cast']=movies['cast'].apply(convert3)
#For movie crew we shall only consider only the job of director and rest shall be ignored
#Function below has been created for the same
def fetch_director(obj):  
  L=[]
  for i in ast.literal_eval(obj):   
   if i['job']=='Director':
    L.append(i['name'])
    break
  return L
movies['crew']=movies['crew'].apply(fetch_director)
#We want to convert overview from string to list so that we can concatenate it with other data
movies['overview']=movies['overview'].apply(lambda x:x.split()) #Lambda function has been used to split the string to a list
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x]) #Lambda funcion has been used to remove white spacing in elements of list genres
#Similar thing shall be done with all the other lists as well
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
#A new list named tags shall be created where all lists above like genres, crew, cast, etc shall be concatenated
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']
#Now we shall create a new data frame consisting only of movie id, title and tags
new_df=movies[['movie_id','title','tags']] #New dataframe has been created
new_df['tags']= new_df['tags'].apply(lambda x:" ".join(x)) #Now we shall form a string out of the tags list by joining all the members of list
new_df['tags']= new_df['tags'].apply(lambda x:x.lower()) #All tags have been converted to lowercase
new_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."
